# 02 - Labeling Intent

---

### Intent Categories

| Intent Label        | Description                                      |
|---------------------|------------------------------------------------|
| `cancel_service`    | Customer wants to cancel account or switch plan|
| `billing_issue`     | Complaints or questions about billing           |
| `technical_issue`   | Problems with device, network, or service       |
| `account_help`      | Issues with account management or login         |
| `upgrade_request`   | Requests to upgrade device or plan               |
| `general_question`  | General questions or inquiries                    |
| `positive_feedback` | Praise or compliments                             |
| `complaint`         | Negative feedback not related to billing or tech|
| `other`             | Anything else or unclear                          |

# Labeling and Logic of Categorization

In [1]:
import pandas as pd
import sys
import os


# Fixing routing issue
project_root = os.path.abspath('..')
sys.path.append(project_root)

from scripts.preprocess import assign_intent

# Loads Data from Cleaned CSV
df = pd.read_csv("../data/processed/cleaned_tweets.csv")

# Sample 250 rows to label
golden_df = df.sample(250, random_state=42).copy()

# Creates Categories to put into column later
intent_categories = [
    "Billing",
    "Technical Support",
    "Account Management",
    "Complaint",
    "Praise/Thank You",
    "Other"
]

keywords = {
    "Billing": ["bill", "charge", "payment", "refund"],
    "Technical Support": ["error", "issue", "problem", "disconnect", "slow"],
    "Account Management": ["password", "login", "account", "reset"],
    "Complaint": ["bad", "terrible", "worst", "disappointed", "angry"],
    "Praise/Thank You": ["thank", "great", "love", "awesome", "appreciate"]
}

# Apply Categorization Function to Datarframe and Write to CSV file

In [2]:
df['intent'] = df['cleaned_text'].apply(lambda text: assign_intent(text, keywords))
print(df[['cleaned_text', 'intent']].sample(10))

df['tweet_length'] = df['cleaned_text'].str.len()

#print(df[['cleaned_text', 'tweet_length']].sample(10))
print(df[['cleaned_text', 'sentiment_score']].sample(10))

# Writes golden dataset to csv file
df.to_csv("../data/processed/golden_intent_labeled.csv", index=False)
print("Golden dataset saved")

                                           cleaned_text             intent
7132   shocked to see a cashier giving out worth of ...              Other
424    i have gb left on my pc amp i have great inte...   Praise/Thank You
9024   thats a great question youre correct until th...   Praise/Thank You
552    hi matthew thank you for loving us despite ou...   Praise/Thank You
4004                                  done thanks a mil   Praise/Thank You
6154   i dont im afraid they dont have a customer se...              Other
654    there is one left by the time i get a refund ...            Billing
4486   we have forwarded your issue to our concern t...  Technical Support
2343   hi toshiaki we had linked the booking to your...   Praise/Thank You
8349   awesome were glad its all sorted out please s...   Praise/Thank You
                                           cleaned_text  sentiment_score
7458                 hey im waitin on dm please help me         0.000000
8976   at least some people u

# Quick Evaluation

In [3]:
# Peep the examples by intent
for label in df['intent'].unique():
    print(f"Intent: {label}")
    sample = df[df['intent'] == label][['cleaned_text']].dropna().sample(3, random_state=1)
    for i, row in enumerate(sample.itertuples(index=False), 1):
        print(f"\n  {i}. {row.cleaned_text}")

Intent: Other

  1.  trying to buy this online but when you click on it the price jumps up why 

  2.  sent to you just now

  3.  you need a fraud reporting email address on your website i should not have to wait on hold for mins to get an email address to report a spoofing email
Intent: Technical Support

  1.  im still having issue with the headquarters and being the only one in it and unable to connect with friends to play as a party

  2.  i was on hold for minutes and you disconnected me about ready to cancel my flight

  3.  we are here to help dm us the details of your issue and we can take a look 
Intent: Praise/Thank You

  1.  whats the source of the mp files is it a screen recording thanks rk

  2.  hello wed like to hear from you help us improve our support by clicking this link thank you

  3.  thanks for giving us a visit today enjoy your cocktail and your flight tbw
Intent: Account Management

  1.  without posting any personal informationaccount details can you elabora

# Additional Features